In [ ]:
from pathlib import Path

import hvplot.polars
import polars as pl

hvplot.extension("bokeh")

In [ ]:
dir = Path(r"Q:\Data\PeerAgencyDashboards\MTC-VitalSigns")
region_df = pl.read_csv(
    dir
    / "Unemployment - Historical Trend for Unemployment Rate - Region - downloaded 240524.csv",
    columns=["year", "unemployment_rate", "region"],
)
county_df = pl.read_csv(
    dir
    / "Unemployment - Historical Trend for Unemployment Rate - Counties - downloaded 240524.csv",
    columns=["year", "unemployment_rate", "county"],
)

In [ ]:
output_df = (
    pl.concat(
        [
            region_df.rename({"region": "geography"}),
            county_df.rename({"county": "geography"}).filter(
                pl.col("geography") == "San Francisco"
            ),
        ]
    )
    .filter(pl.col("year") > 2007)
    .select(
        "year",
        "geography",
        (pl.col("unemployment_rate") * 100).alias("unemployment rate (%)"),
    )
)
output_df.write_csv("output/csvs/unemployment.csv")
plot = output_df.plot.line(
    x="year",
    y="unemployment rate (%)",
    by="geography",
    xticks=output_df.select(pl.col("year").unique()).to_series().to_list(),
)
hvplot.save(plot, "output/figs/unemployment.png")
plot
